Select up to 4 atoms to get internal coordinates

Use 2 atoms to get their bond length, 3 to get the angle and 4 to get the dihedral angle

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import Layout, Dropdown, Button, Output, Checkbox, Label
from IPython.display import display, clear_output
from __future__ import print_function
import json
from IPython.display import display
from tools import simplify_smiles
from ipywidgets import GridspecLayout
import os

In [3]:
def split_atoms(s, multi_letter_atoms):
    atoms = []
    i = 0
    while i < len(s):
        if i + 1 < len(s) and s[i:i+2] in multi_letter_atoms:
            atoms.append(s[i:i+2])
            i += 2
        else:
            atoms.append(s[i])
            i += 1
    return atoms

def canonical_key(selected_atoms, multi_letter_atoms):
    atoms_fwd = split_atoms(selected_atoms, multi_letter_atoms)
    atoms_rev = list(reversed(atoms_fwd))
    key_fwd = ''.join(atoms_fwd)
    key_rev = ''.join(atoms_rev)
    return min(key_fwd, key_rev)

In [5]:
def load_cleaned_dicts(folder="Cleaned"):
    """
    Loads each cleaned JSON file in the specified folder and its 'Geometries' subfolder
    into a separate variable-like dictionary, matching the variable names used in your code.
    Returns a dictionary with these variable names as keys and the loaded dictionaries as values.
    """
    cleaned_dicts = {}

    # Main folder files
    for filename in os.listdir(folder):
        if filename.endswith(".json"):
            varname = filename.replace(".json", "")
            filepath = os.path.join(folder, filename)
            with open(filepath, "r") as f:
                cleaned_dicts[varname] = json.load(f)
    return cleaned_dicts
"""
    # Geometries subfolder files
    geometries_folder = os.path.join(folder, "Geometries")
    if os.path.isdir(geometries_folder):
        for geo in os.listdir(geometries_folder):
            geo_path = os.path.join(geometries_folder, geo)
            if os.path.isdir(geo_path):
                for filename in os.listdir(geo_path):
                    if filename.endswith(".json"):
                        # e.g. bonds.json -> cleaned_GOAT_bonds_by_geo_Oktaedrisch
                        base = filename.replace('.json', '')
                        varname = f"cleaned_GOAT_{base}_by_geo_{geo}"
                        filepath = os.path.join(geo_path, filename)
                        with open(filepath, "r") as f:
                            cleaned_dicts[varname] = json.load(f)
"""

# Load all cleaned dictionaries
all_cleaned_dicts = load_cleaned_dicts()

# Assign to variables for use in the rest of the program
cleaned_GOAT_bonds = all_cleaned_dicts.get("cleaned_GOAT_bonds", {})
cleaned_GOAT_angles = all_cleaned_dicts.get("cleaned_GOAT_angles", {})
cleaned_GOAT_torsions = all_cleaned_dicts.get("cleaned_GOAT_torsions", {})
cleaned_CCDC_bonds = all_cleaned_dicts.get("cleaned_CCDC_bonds", {})
cleaned_CCDC_angles = all_cleaned_dicts.get("cleaned_CCDC_angles", {})
cleaned_CCDC_torsions = all_cleaned_dicts.get("cleaned_CCDC_torsions", {})
cleaned_HCSD_bonds = all_cleaned_dicts.get("cleaned_HCSD_bonds", {})
cleaned_HCSD_angles = all_cleaned_dicts.get("cleaned_HCSD_angles", {})
cleaned_HCSD_torsions = all_cleaned_dicts.get("cleaned_HCSD_torsions", {})
cleaned_OMol25_bonds = all_cleaned_dicts.get("cleaned_OMol25_bonds", {})
"""
# Geometry-specific dictionaries
geometries = [
    "Oktaedrisch", "Quadratisch_Planar", "Quadratisch_Pyramidal", "T_Foermig",
    "Tetraedrisch", "Trigonal_Bipyramidal", "Trigonal_Planar", "Trigonal_Pyramidal"
]
cleaned_GOAT_bonds_by_geo = {}
cleaned_GOAT_angles_by_geo = {}
cleaned_GOAT_torsions_by_geo = {}
for geo in geometries:
    cleaned_GOAT_bonds_by_geo[geo] = all_cleaned_dicts.get(f"cleaned_GOAT_bonds_by_geo_{geo}", {})
    cleaned_GOAT_angles_by_geo[geo] = all_cleaned_dicts.get(f"cleaned_GOAT_angles_by_geo_{geo}", {})
    cleaned_GOAT_torsions_by_geo[geo] = all_cleaned_dicts.get(f"cleaned_GOAT_torsions_by_geo_{geo}", {})

# Now you can use these variables as before in your program!
"""

'\n# Geometry-specific dictionaries\ngeometries = [\n    "Oktaedrisch", "Quadratisch_Planar", "Quadratisch_Pyramidal", "T_Foermig",\n    "Tetraedrisch", "Trigonal_Bipyramidal", "Trigonal_Planar", "Trigonal_Pyramidal"\n]\ncleaned_GOAT_bonds_by_geo = {}\ncleaned_GOAT_angles_by_geo = {}\ncleaned_GOAT_torsions_by_geo = {}\nfor geo in geometries:\n    cleaned_GOAT_bonds_by_geo[geo] = all_cleaned_dicts.get(f"cleaned_GOAT_bonds_by_geo_{geo}", {})\n    cleaned_GOAT_angles_by_geo[geo] = all_cleaned_dicts.get(f"cleaned_GOAT_angles_by_geo_{geo}", {})\n    cleaned_GOAT_torsions_by_geo[geo] = all_cleaned_dicts.get(f"cleaned_GOAT_torsions_by_geo_{geo}", {})\n\n# Now you can use these variables as before in your program!\n'

In [6]:
def adjust_string_for_special_cases(s, special_cases):
    # Replace each special case with a single character (e.g., a marker)
    for case in special_cases:
        s = s.replace(case, "X")  # Assuming "X" does not appear in your string
    return s

def get_adjusted_length(s, special_cases):
    adjusted_s = adjust_string_for_special_cases(s, special_cases)
    return len(adjusted_s)

def select_dictionaries(selected_atoms, selected_geometry):
    chosen_dict = {}
    chosen_dict2 = {}
    chosen_dict_own = {}

    multi_letter_atoms = ['Cl', 'Br', 'Sc', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
'La', 'Hf', 'Ta', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg']

    adjusted_length = get_adjusted_length(selected_atoms, multi_letter_atoms)
    
    # GOAT-Daten je nach Auswahl
    if selected_geometry == "Alle":
        goat_bonds = cleaned_GOAT_bonds
        goat_angles = cleaned_GOAT_angles
        goat_torsions = cleaned_GOAT_torsions
    if adjusted_length == 2:
        chosen_dict_own = goat_bonds
    elif adjusted_length == 3:
        chosen_dict_own = goat_angles
    elif adjusted_length == 4:
        chosen_dict_own = goat_torsions
    else:
        print("Invalid number of atoms")

    atoms_list = split_atoms(selected_atoms, multi_letter_atoms)
    has_hydrogen = "H" in atoms_list
    # Check if "H" is in selected_atoms
    if has_hydrogen:
        if adjusted_length == 2:
            chosen_dict = cleaned_HCSD_bonds
        elif adjusted_length == 3:
            chosen_dict = cleaned_HCSD_angles
        elif adjusted_length == 4:
            chosen_dict = cleaned_HCSD_torsions
        else:
            print("Invalid number of atoms")
    else:
    # If "H" is not in selected_atoms, use the dictionaries without "H"
        if adjusted_length == 2:
            chosen_dict = cleaned_CCDC_bonds
        elif adjusted_length == 3:
            chosen_dict = cleaned_CCDC_angles
        elif adjusted_length == 4:
            chosen_dict = cleaned_CCDC_torsions
        else:
            print("Invalid number of atoms")
    if adjusted_length == 2:
        chosen_dict2 = cleaned_OMol25_bonds       
    """
    elif adjusted_length == 3:
        chosen_dict = cleaned_CCDC_angles
    elif adjusted_length == 4:           
        chosen_dict = cleaned_CCDC_torsions
    else:
        print("Invalid number of atoms")
    """        
    return chosen_dict, chosen_dict2, chosen_dict_own

def draw_plot(selected_atoms, selected_geometry, chosen_dict, chosen_dict2, chosen_dict_own, fig, index):
    multi_letter_atoms = ['Cl', 'Br', 'Sc', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'La', 'Hf', 'Ta', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg']
    selected_atoms_list = split_atoms(selected_atoms, multi_letter_atoms)
    key = canonical_key(selected_atoms, multi_letter_atoms)

    CCDC_data = chosen_dict.get(key)
    xyz_data = chosen_dict_own.get(key)
    OMol25_data = chosen_dict2.get(key) if chosen_dict2 else None

    if xyz_data is None or len(xyz_data) == 0:
        print(f"{selected_atoms} not found or empty in GOAT!")
        return "not found"

    if (CCDC_data is None or len(CCDC_data) == 0) and (OMol25_data is None or len(OMol25_data)== 0):
        print(f"{selected_atoms} not found or empty in both CCDC and OMol25!")
        return "not found"

    print("Internal Coordinates:", selected_atoms)

    import numpy as np

    # Nach print("Internal Coordinates:", selected_atoms)

    # Umwandlung in numpy arrays für mathematische Operationen
    CCDC_data = np.array(CCDC_data) if CCDC_data is not None and len(CCDC_data) > 0 else None
    xyz_data = np.array(xyz_data) if xyz_data is not None and len(xyz_data) > 0 else None
    OMol25_data = np.array(OMol25_data) if OMol25_data is not None and len(OMol25_data) > 0 else None

    # Sammle alle vorhandenen Daten für die gemeinsame Range
    all_data = []
    # ---------------------------------------------
    # OMol25_data = None
    # ---------------------------------------------
    if CCDC_data is not None:
        all_data.append(CCDC_data)
    if xyz_data is not None:
        all_data.append(xyz_data)
    if OMol25_data is not None:
        all_data.append(OMol25_data)

    if not all_data:
        print(f"{selected_atoms} keine Daten zum Plotten vorhanden!")
        return "not found"

    all_data_concat = np.concatenate(all_data)
    data_min = np.percentile(all_data_concat, 2.5)
    data_max = np.percentile(all_data_concat, 97.5)
    data_min = max(data_min-0.2, 0.0)  # Ensure minimum is not negative
    data_max = data_max + 0.2  # Add a small buffer to the maximum
    print(f"Data Range: {data_min} to {data_max}")

    # Neue Zeilen: Limit auf 5 Å für Bindungslängen (2 Atome)
    if len(selected_atoms_list) == 2:  # Nur für Bindungslängen
        data_max = min(data_max, 5.0)

    bins = 100

    n_CCDC = len(CCDC_data) if CCDC_data is not None else 0
    n_GOAT = len(xyz_data) if xyz_data is not None else 0
    n_OMol25 = len(OMol25_data) if OMol25_data is not None else 0

    print(f"CSD: {n_CCDC}, GOAT: {n_GOAT}, OMol25: {n_OMol25}")
    axs = fig.add_subplot(2,2,index)

    # CCDC
    if CCDC_data is not None:
        counts_CCDC, bins_CCDC = np.histogram(CCDC_data, bins=bins, range=(data_min, data_max))
        counts_CCDC_norm = counts_CCDC / counts_CCDC.sum() if counts_CCDC.sum() > 0 else counts_CCDC
        axs.bar(bins_CCDC[:-1], counts_CCDC_norm, width=bins_CCDC[1]-bins_CCDC[0], color="skyblue", edgecolor="black", label="CSD Data")
    else:
        bins_CCDC = np.linspace(data_min, data_max, bins + 1)

    
    # GOAT
    if selected_geometry == "Alle":
        goat_label = "GOAT Data"
    else:
        goat_label = f"GOAT Data ({selected_geometry.replace('_', ' ')})"

    if xyz_data is not None:
        counts_GOAT, bins_GOAT = np.histogram(xyz_data, bins=bins_CCDC)
        counts_GOAT_norm = counts_GOAT / counts_GOAT.sum() if counts_GOAT.sum() > 0 else counts_GOAT
        axs.bar(bins_GOAT[:-1], counts_GOAT_norm, width=bins_GOAT[1]-bins_GOAT[0], color="pink", edgecolor="black", 
                label=goat_label, alpha=0.5)

    # OMol25 (nur bei 2 Atomen)
    if OMol25_data is not None and len(selected_atoms_list) == 2:
        counts_OMol25, bins_OMol25 = np.histogram(OMol25_data, bins=bins_CCDC)
        counts_OMol25_norm = counts_OMol25 / counts_OMol25.sum() if counts_OMol25.sum() > 0 else counts_OMol25
        axs.bar(
            bins_OMol25[:-1],
            counts_OMol25_norm,
            width=bins_OMol25[1]-bins_OMol25[0],
            color="orange",
            edgecolor="black",
            label="OMol25 Data",
            alpha=0.5
        )

    axs.legend(loc="upper right")
    label_atoms = '-'.join(selected_atoms_list)
    label_bonds = f"{label_atoms} Bond Length / Å "
    label_angles = f"{label_atoms} Angle / $\degree$"
    label_dihedrals = f"{label_atoms} Dihedral /$\degree$"
    label_ybonds = r"relative frequency"
    label_yangles = r"relative frequency"
    label_ytorsion = r"relative frequency"

    # choose the label based on the chosen dictionary
    if len(selected_atoms_list) == 2:
        chosen_label_x = label_bonds
        chosen_label_y = label_ybonds
    elif len(selected_atoms_list) == 3:
        chosen_label_x = label_angles
        chosen_label_y = label_yangles
    elif len(selected_atoms_list) == 4:
        chosen_label_x = label_dihedrals
        chosen_label_y = label_ytorsion
    else:
        print("Invalid dictionary")

    axs.set_xlabel(chosen_label_x)
    axs.set_ylabel(chosen_label_y)
    # ...in draw_plot(), nach dem letzten axs.bar(...):

    # Sammle alle Normierungswerte
    y_max = 0
    if CCDC_data is not None:
        y_max = max(y_max, counts_CCDC_norm.max())
    if xyz_data is not None:
        y_max = max(y_max, counts_GOAT_norm.max())
    if OMol25_data is not None and len(selected_atoms_list) == 2:
        y_max = max(y_max, counts_OMol25_norm.max())

    axs.set_ylim(0, y_max + 0.1)



In [ ]:
# Add this new cell:

import matplotlib.pyplot as plt
import numpy as np
import os

def create_publication_plot(selected_atoms, output_dir="Publication_Plots"):
    """
    Create a publication-ready plot with specific formatting requirements
    """
    # Publication formatting settings
    FONT_SIZE = 12  # = 15pt
    SCALE = 1
    SCALED_FONT_SIZE = FONT_SIZE  # = 15pt
    
    # Set font sizes to achieve 12pt after 0.8\textwidth scaling
    plt.rcParams.update({
        'font.family': 'serif',
        'font.serif': ['Times New Roman', 'Times', 'DejaVu Serif'],
        'font.size': SCALED_FONT_SIZE,
        'axes.titlesize': SCALED_FONT_SIZE,
        'axes.labelsize': SCALED_FONT_SIZE,
        'xtick.labelsize': SCALED_FONT_SIZE,
        'ytick.labelsize': SCALED_FONT_SIZE,
        'legend.fontsize': SCALED_FONT_SIZE
    })
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Setup data
    multi_letter_atoms = ['Cl', 'Br', 'Sc', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Zr', 'Nb', 'Mo', 'Tc', 
                          'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'La', 'Hf', 'Ta', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg']
    selected_geometry = "Alle"
    
    # Get canonical key and dictionaries
    key = canonical_key(selected_atoms, multi_letter_atoms)
    chosen_dict, chosen_dict2, chosen_dict_own = select_dictionaries(selected_atoms, selected_geometry)
    selected_atoms_list = split_atoms(selected_atoms, multi_letter_atoms)
    
    # Get data
    CCDC_data = chosen_dict.get(key)
    xyz_data = chosen_dict_own.get(key)
    OMol25_data = chosen_dict2.get(key) if chosen_dict2 else None
    
    # Check if we have data
    if xyz_data is None or len(xyz_data) == 0:
        print(f"{selected_atoms} not found or empty in GOAT!")
        return False
        
    if (CCDC_data is None or len(CCDC_data) == 0) and (OMol25_data is None or len(OMol25_data) == 0):
        print(f"{selected_atoms} not found or empty in both CCDC and OMol25!")
        return False
    
    # Convert to numpy arrays
    CCDC_data = np.array(CCDC_data) if CCDC_data is not None and len(CCDC_data) > 0 else None
    xyz_data = np.array(xyz_data) if xyz_data is not None and len(xyz_data) > 0 else None
    OMol25_data = np.array(OMol25_data) if OMol25_data is not None and len(OMol25_data) > 0 else None
    
    # Calculate data range
    all_data = []
    if CCDC_data is not None:
        all_data.append(CCDC_data)
    if xyz_data is not None:
        all_data.append(xyz_data)
    if OMol25_data is not None:
        all_data.append(OMol25_data)
        
    if not all_data:
        print(f"{selected_atoms} keine Daten zum Plotten vorhanden!")
        return False
        
    all_data_concat = np.concatenate(all_data)
    data_min = np.percentile(all_data_concat, 2.5)
    data_max = np.percentile(all_data_concat, 97.5)
    data_min = max(data_min-0.2, 0.0)
    data_max = data_max + 0.2
    
    # Limit bond lengths to 5 Å
    if len(selected_atoms_list) == 2:
        data_max = min(data_max, 5.0)
    
    # Create figure with specific size
    fig = plt.figure(figsize=(5.927*SCALE, 3.7*SCALE))
    ax = fig.add_subplot(1, 1, 1)
    
    bins = 100
    
    # Plot data
    if CCDC_data is not None:
        counts_CCDC, bins_CCDC = np.histogram(CCDC_data, bins=bins, range=(data_min, data_max))
        counts_CCDC_norm = counts_CCDC / counts_CCDC.sum() if counts_CCDC.sum() > 0 else counts_CCDC
        ax.bar(bins_CCDC[:-1], counts_CCDC_norm, width=bins_CCDC[1]-bins_CCDC[0], 
               color="skyblue", edgecolor="black", label="CSD Data", alpha=0.7)
    else:
        bins_CCDC = np.linspace(data_min, data_max, bins + 1)
    
    if xyz_data is not None:
        counts_GOAT, bins_GOAT = np.histogram(xyz_data, bins=bins_CCDC)
        counts_GOAT_norm = counts_GOAT / counts_GOAT.sum() if counts_GOAT.sum() > 0 else counts_GOAT
        ax.bar(bins_GOAT[:-1], counts_GOAT_norm, width=bins_GOAT[1]-bins_GOAT[0], 
               color="pink", edgecolor="black", label="GOAT Data", alpha=0.7)
    
    if OMol25_data is not None and len(selected_atoms_list) == 2:
        counts_OMol25, bins_OMol25 = np.histogram(OMol25_data, bins=bins_CCDC)
        counts_OMol25_norm = counts_OMol25 / counts_OMol25.sum() if counts_OMol25.sum() > 0 else counts_OMol25
        ax.bar(bins_OMol25[:-1], counts_OMol25_norm, width=bins_OMol25[1]-bins_OMol25[0],
               color="orange", edgecolor="black", label="OMol25 Data", alpha=0.7)
    
    # Labels
    label_atoms = '-'.join(selected_atoms_list)
    if len(selected_atoms_list) == 2:
        ax.set_xlabel(f"{label_atoms} Bond Length / Å")
        ax.set_ylabel("Relative Frequency")
    elif len(selected_atoms_list) == 3:
        ax.set_xlabel(f"{label_atoms} Angle / °")
        ax.set_ylabel("Relative Frequency")
    elif len(selected_atoms_list) == 4:
        ax.set_xlabel(f"{label_atoms} Dihedral / °")
        ax.set_ylabel("Relative Frequency")
    
    # Set y-axis limits
    y_max = 0
    if CCDC_data is not None:
        y_max = max(y_max, counts_CCDC_norm.max())
    if xyz_data is not None:
        y_max = max(y_max, counts_GOAT_norm.max())
    if OMol25_data is not None and len(selected_atoms_list) == 2:
        y_max = max(y_max, counts_OMol25_norm.max())
    
    ax.set_ylim(0, y_max + 0.1*y_max)
    
    # Legend
    ax.legend(loc="upper right")
    
    # Tight layout
    plt.tight_layout()
    
    # Save plot
    # Determine metal and subfolder for organization
    metals = ['Sc', 'Ti', 'V', 'Cr', 'Mn', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Hf', 'Ta', 'W', 'Re']
    metal = get_metal_from_combo(key, metals)
    adjusted_length = get_adjusted_length(key, multi_letter_atoms)
    
    if adjusted_length == 2:
        subfolder = "Bonds"
    elif adjusted_length == 3:
        subfolder = "Angles"
    elif adjusted_length == 4:
        subfolder = "Torsions"
    else:
        subfolder = "Other"
    
    # Create subdirectories
    metal_dir = os.path.join(output_dir, metal, subfolder)
    os.makedirs(metal_dir, exist_ok=True)
    
    # Save as both PNG and PDF
    base_filename = os.path.join(metal_dir, key)
    plt.savefig(f"{base_filename}.png", dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.savefig(f"{base_filename}.pdf", bbox_inches='tight', pad_inches=0.1)
    
    plt.close(fig)
    
    print(f"Publication plot saved: {base_filename}.png/.pdf")
    return True

# Function to create publication plots for specific combinations
def create_publication_plots_batch(atom_combinations, output_dir="Publication_Plots"):
    """
    Create publication plots for a list of atom combinations
    
    Parameters:
    atom_combinations: list of strings like ["CScN", "CScO", "NScO"]
    output_dir: output directory for plots
    """
    successful = 0
    failed = 0
    
    for atoms in atom_combinations:
        try:
            if create_publication_plot(atoms, output_dir):
                successful += 1
            else:
                failed += 1
        except Exception as e:
            print(f"Error creating plot for {atoms}: {e}")
            failed += 1
    
    print(f"\nPublication plot batch complete:")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    
    return successful, failed

# Example usage:
# create_publication_plot("CScN")
# create_publication_plots_batch(["CScN", "CScO", "NScO", "CScS"])

In [ ]:

atoms = ["-", "C", "H", "N", "S", "O", "P", "Cl", 'Sc', 'Ti', 'V', 'Cr', 'Mn',
'Y', 'Zr', 'Nb', 'Mo', 'Tc',
'La', 'Hf', 'Ta', 'W', 'Re']


def on_clear_click(b):
    for i in range(4):
        if grid[i,grid.n_columns-1] is b:
            for j in range(1, 5):
                grid[i, j].value = "-"   
                
def on_button_click(b):
    
    b.description = "Update"
    
    coordinates1 = [grid[0,x+1].value for x in range(grid.n_columns-2)]
    coordinates2 = [grid[1,x+1].value for x in range(grid.n_columns-2)]
    coordinates3 = [grid[2,x+1].value for x in range(grid.n_columns-2)]
    coordinates4 = [grid[3,x+1].value for x in range(grid.n_columns-2)]


    coordinates1 = ["" if value == "-" else value for value in coordinates1]
    selected_atoms1 = "".join(coordinates1)
    coordinates2 = ["" if value == "-" else value for value in coordinates2]
    selected_atoms2 = "".join(coordinates2)
    coordinates3 = ["" if value == "-" else value for value in coordinates3]
    selected_atoms3 = "".join(coordinates3)
    coordinates4 = ["" if value == "-" else value for value in coordinates4]
    selected_atoms4 = "".join(coordinates4)
    
    selected_geometry = "Alle"

    with output:
        clear_output()
        fig = plt.figure()
        fig.tight_layout()
        fig.set_figheight(10)
        fig.set_figwidth(15)
        if selected_atoms1 != "":
            chosen_dict1, chosen_dict2_1, chosen_dict_own1 = select_dictionaries(selected_atoms1, selected_geometry)
            draw_plot(selected_atoms1,selected_geometry, chosen_dict1, chosen_dict2_1, chosen_dict_own1, fig, 1)
        if selected_atoms2 != "":
            chosen_dict2, chosen_dict2_2, chosen_dict_own2 = select_dictionaries(selected_atoms2, selected_geometry)
            draw_plot(selected_atoms2,selected_geometry,chosen_dict2, chosen_dict2_2, chosen_dict_own2, fig, 2)
        if selected_atoms3 != "":
            chosen_dict3, chosen_dict2_3, chosen_dict_own3 = select_dictionaries(selected_atoms3, selected_geometry)
            draw_plot(selected_atoms3,selected_geometry, chosen_dict2_3, chosen_dict3, chosen_dict_own3, fig, 3)
        if selected_atoms4 != "":
            chosen_dict4, chosen_dict2_4, chosen_dict_own4 = select_dictionaries(selected_atoms4, selected_geometry)
            draw_plot(selected_atoms4,selected_geometry, chosen_dict2_4, chosen_dict4, chosen_dict_own4, fig, 4)
        plt.show()
    if check.value:
        # Collect non-empty selected atoms strings
        selected_atoms = [selected_atoms1, selected_atoms2, selected_atoms3, selected_atoms4]
        internal_coordinates = [atom for atom in selected_atoms if atom != ""]

        # Use these strings to create a filename
        safe_title = "_".join(internal_coordinates)  # create a safe title with only the internal coordinates
        fig.savefig(f"{safe_title}.pdf", format="pdf", transparent=True, bbox_inches='tight', pad_inches=0)  # save the plot with the title as the filename

grid = GridspecLayout(4, 6, layout=Layout(width="700px"))

for i in range(4):
    for j in range(6):
        if j == 0:
            grid[i, j] = Label(f"Atoms for Plot {i+1}")
        elif j == 5:
            clear_button = Button(description="Clear", layout=Layout(width="100px"))
            clear_button.on_click(on_clear_click)
            grid[i, j] = clear_button
        else:
            grid[i, j] = Dropdown(options=atoms, layout=Layout(width="100px"))
    

display(grid)

check = Checkbox(value=False, description="Save Plots") # checkbox to save the plot
display(check)

output = Output()
display(output)

     
button = Button(description='Start')
button.on_click(on_button_click)
display(button)
